This example is based on the <a href="https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html">Comparing different clustering algorithms on toy datasets</a> example from the scikit-learn documentation.  Where possible, it has been modified to use GapStatClustering to estimate the number of custers rather than accepting the number of clusters as a given input parameter.

In [ ]:
#print(__doc__)

%matplotlib inline

import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

from gapstat import GapStatClustering

In [ ]:
data = {
    "noisy_circles": {
        "data": None,
        "params": None,
        "times": {},
        "results": {}
    },
    "noisy_moons": {
        "data": None,
        "params": None,
        "times": {},
        "results": {}
    },
    "blobs": {
        "data": None,
        "params": None,
        "times": {},
        "results": {}
    },
    "no_structure": {
        "data": None,
        "params": None,
        "times": {},
        "results": {}
    },
    "aniso": {
        "data": None,
        "params": None,
        "times": {},
        "results": {}
    },
    "varied": {
        "data": None,
        "params": None,
        "times": {},
        "results": {}        
    }
}

In [ ]:
# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
np.random.seed(0)

n_samples = 1500

data["noisy_circles"]["data"] = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)
data["noisy_circles"]["params"] = {'damping': .77, 'preference': -240,
                                   'quantile': .2, 'n_clusters': 2,
                                   'min_samples': 20, 'xi': 0.25}

data["noisy_moons"]["data"] = datasets.make_moons(n_samples=n_samples, noise=.05)
data["noisy_moons"]["params"] = {'damping': .75, 'preference': -220, 'n_clusters': 2}

data["blobs"]["data"] = datasets.make_blobs(n_samples=n_samples, random_state=8)
data["blobs"]["params"] = {}

data["no_structure"]["data"] = np.random.rand(n_samples, 2), None
data["no_structure"]["params"] = {}

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
data["aniso"]["data"] = (X_aniso, y)
data["aniso"]["params"] = {'eps': .15, 'n_neighbors': 2,
                           'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}

# blobs with varied variances
data["varied"]["data"] = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)
data["varied"]["params"] = {'eps': .18, 'n_neighbors': 2,
                            'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}

# ============
# Default cluster parameters
# ============
default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

In [ ]:
# ============
# Cluster the datasets using different clustering algorithms
# - use GapStatClustering when possible
# ============
for dataset_name in data:
    print(dataset_name)

    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(data[dataset_name]["params"])

    X, y = data[dataset_name]["data"]

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects and wrap with GapStatClustering when possible
    # ============
    # MeanShift is not compatible with GapStatClustering
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    # MiniBatchKMeans without GapStatClustering
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    # MiniBatchKMeans with GapStatClustering
    two_means_gs = GapStatClustering(base_clusterer=cluster.MiniBatchKMeans())
    # Ward Linkage (AgglomerativeClustering) doesn't work reliably with GapStatClustering
    # - need to bump max_k up to a large value - need to figure out why
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    # SpectralClustering without GapStatClustering 
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    # SpectralClustering with GapStatClustering 
    spectral_gs = GapStatClustering(base_clusterer=cluster.SpectralClustering(
        eigen_solver='arpack',
        affinity="nearest_neighbors"))
    # DBSCAN is not compatible wth GapStatClustering
    dbscan = cluster.DBSCAN(eps=params['eps'])
    # OPTICS is not compatible wth GapStatClustering
    optics = cluster.OPTICS(min_samples=params['min_samples'],
                            xi=params['xi'],
                            min_cluster_size=params['min_cluster_size'])
    # AffinityPropagation is not compatible wth GapStatClustering 
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    # Average Linkage (AgglomerativeClustering) without GapStatClustering 
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    # Average Linkage (AgglomerativeClustering) with GapStatClustering 
    average_linkage_gs = GapStatClustering(base_clusterer=cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        connectivity=connectivity))
    # Birch doesn't work reliably with GapStatClustering
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    # GaussianMixture is not currently comptible with GapStatClusterig
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('MiniBatchKMeans (GS)', two_means_gs),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('Spectral', spectral),
        ('Spectral (GS)', spectral_gs),
        ('Ward', ward),
        ('Agglomerative', average_linkage),
        ('Agglomerative (GS)', average_linkage_gs),
        ('DBSCAN', dbscan),
        ('OPTICS', optics),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

    for name, algorithm in clustering_algorithms:
        print("-- " + name)
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        
        # add run time to dataset
        data[dataset_name]["times"][name] = t1 - t0

        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)
        
        # add results to dataset
        data[dataset_name]["results"][name] = y_pred

In [ ]:
# ============
# Plot the results
# ============
fig, axs = plt.subplots(len(data), len(clustering_algorithms), figsize=(20, 10))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

subplot_row = 0
for dataset_name in data:
    X, y = data[dataset_name]["data"]
    
    subplot_col = 0
    for algo_name, _ in clustering_algorithms:
        y_pred = data[dataset_name]["results"][algo_name]
        runtime = data[dataset_name]["times"][algo_name]

        if subplot_row == 0:
            axs[subplot_row, subplot_col].set_title(algo_name, size=10)

        if subplot_col == 0:
            axs[subplot_row, subplot_col].set_ylabel(dataset_name, size=10)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])

        axs[subplot_row, subplot_col].scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        #axs[subplot_row, subplot_col].set_xlim(-2.5, 2.5)
        #axs[subplot_row, subplot_col].set_ylim(-2.5, 2.5)
        axs[subplot_row, subplot_col].set_xticks(())
        axs[subplot_row, subplot_col].set_yticks(())
        time_text = ('%.2fs' % (runtime)).lstrip('0')
        axs[subplot_row, subplot_col].text(0.99, 0.01, time_text, horizontalalignment='right',
                                           size=15,
                                           transform=axs[subplot_row, subplot_col].transAxes)

        subplot_col += 1
    subplot_row += 1